In [1]:
%load_ext autoreload
%autoreload 2
#%reload_ext autoreload

In [12]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import Generator as Generator 
import model_definitions

### Open input file

In [24]:
SEQUENCE_LENGTH=20
ONE_CAR=True
PATH=r"D:\Users\Students\Daniele\records\OneCar_db.txt"
BATCH_SIZE=256*2
FEATURES=7 
FEATURES_PREDICTED=3

In [25]:
maxSpeed=40
frequency=0.02
maxdiff=maxSpeed*frequency
maxAngularVelocity=140 #140 degrees for second
maxAngDiff=maxAngularVelocity*frequency
print(f"Max speed diff: {maxdiff}\nMax x/z diff: {maxdiff}\nMax rot diff: {maxAngDiff}")

Max speed diff: 0.8
Max x/z diff: 0.8
Max rot diff: 2.8000000000000003


In [ ]:
Generator.SEQUENCE_LENGTH=SEQUENCE_LENGTH
Generator.FEATURES=FEATURES
df=Generator.single_care_dataframe(PATH)

In [ ]:
#def SubtractDF(df):
#    return df.groupby("RACE",group_keys=False).apply(Generator.subtraction_columns)

dfs=[]
for i in range(len(df)):
    #dfs.append(SubtractDF(df[i]))
    dfs.append(df[i])

united=dfs[0]
for i in range(1,len(df)):
    united=pd.concat([united,dfs[i] ],ignore_index=True)

In [ ]:
united.tail()

In [ ]:
df_train,df_val,df_test= Generator.split_train_validation_test(united,["RACE","GROUP"])

In [ ]:
df_train.head()

In [ ]:
gen_train=Generator.DataGenerator(BATCH_SIZE,df_train,16000)
gen_val=Generator.DataGenerator(BATCH_SIZE,df_val,16000)
gen_test=Generator.DataGenerator(BATCH_SIZE,df_test,16000)


### Model definition

#### Train and print

In [ ]:
if(ONE_CAR):
    model_definitions.cars=1
else:
    model_definitions.cars=4
model_definitions.features=FEATURES
model_definitions.sequence_length=SEQUENCE_LENGTH
model_definitions.feat_pred=FEATURES_PREDICTED

### Rules Model

In [ ]:
lstm,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.01,epoch=100, dropout=0.2, cells=6 )

ev=lstm.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
lstm2,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.002,epoch=100, dropout=0.02, cells=1)

ev=lstm2.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
mlp,history=MLP_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100)

ev=mlp.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
siamese,history=LSTM_Siamese_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100, dropout=0.02, cells=2 )

ev=siamese.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

### Physichs Model

Since physichs data can be very small, Each data is multiply by 10 

In [11]:
mlp_ph,history_mlp_ph=model_definitions.MLP_Train(gen_train,gen_val,learning_rate=0.0001,epoch=10,verbose=1,name="MLP",path="logs")

ev=mlp_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
tf.saved_model.save(mlp_ph,f'saved_model/MLP0001_PosAssoluta')

Epoch 1/10
5158/5158 [==============================] - 496s 96ms/step - loss: 14.2417 - mean_absolute_error: 1.8190 - accuracy: 1.2622e-07 - val_loss: 1.2071 - val_mean_absolute_error: 0.7566 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/10
5158/5158 [==============================] - 496s 96ms/step - loss: 0.5284 - mean_absolute_error: 0.4881 - accuracy: 6.3110e-07 - val_loss: 0.2291 - val_mean_absolute_error: 0.3358 - val_accuracy: 1.0153e-06 - lr: 1.0000e-04
Epoch 3/10
5158/5158 [==============================] - 500s 97ms/step - loss: 0.1611 - mean_absolute_error: 0.2807 - accuracy: 1.7671e-06 - val_loss: 0.1318 - val_mean_absolute_error: 0.2546 - val_accuracy: 2.2844e-06 - lr: 1.0000e-04
Epoch 4/10
5158/5158 [==============================] - 500s 97ms/step - loss: 0.1165 - mean_absolute_error: 0.2401 - accuracy: 2.1457e-06 - val_loss: 0.1093 - val_mean_absolute_error: 0.2323 - val_accuracy: 1.2691e-06 - lr: 1.0000e-04
Epoch 5/10
5158/5158 [==============================] -

In [13]:
units=32
kernel_size=1
filters=32
normalization=False
cnn_lstm,history_cnn_lstm=model_definitions.CNN_LSTM_Train(gen_train,gen_val,learning_rate=0.0001,patience=10,name=f"CNN{filters}_{kernel_size}LSTM{units}_Norm{normalization}",
                                                           path="logs_350SequenceLength",epoch=10, dropout=0.2,units=units,kernel_size=kernel_size,
                                                           filters=filters, normalization=normalization,verbose=1
                                                          )

ev=cnn_lstm.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

tf.saved_model.save(cnn_lstm,f'saved_model/CNN_LSTM{units}_{kernel_size}_{filters}_PosAssoluta')

Epoch 1/10
5655/5655 [==============================] - 2340s 413ms/step - loss: 0.9373 - mean_absolute_error: 0.6307 - accuracy: 4.6051e-08 - val_loss: 0.7536 - val_mean_absolute_error: 0.5764 - val_accuracy: 9.2807e-08 - lr: 1.0000e-04
Epoch 2/10
5655/5655 [==============================] - 2301s 407ms/step - loss: 0.5683 - mean_absolute_error: 0.5047 - accuracy: 0.0000e+00 - val_loss: 0.4222 - val_mean_absolute_error: 0.4416 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 3/10
5655/5655 [==============================] - 2333s 412ms/step - loss: 0.3508 - mean_absolute_error: 0.4066 - accuracy: 0.0000e+00 - val_loss: 0.2898 - val_mean_absolute_error: 0.3652 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 4/10
5655/5655 [==============================] - 2304s 407ms/step - loss: 0.2785 - mean_absolute_error: 0.3564 - accuracy: 0.0000e+00 - val_loss: 0.2501 - val_mean_absolute_error: 0.3341 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 5/10
5655/5655 [==========================

INFO:tensorflow:Assets written to: saved_model/CNN_LSTM32_1_32_PosAssoluta\assets


INFO:tensorflow:Assets written to: saved_model/CNN_LSTM32_1_32_PosAssoluta\assets


In [23]:
lr =1e-4
units=8
cells=1
normalization=False
name=f"LSTM{cells}_{units}_{lr}_Norm{normalization}"
lstm_ph,history_lstm_ph=model_definitions.LSTM_Train(gen_train,gen_val,learning_rate=lr,patience=6,name=name,path="logs",
                                                     epoch=10, dropout=0.2, cells=cells,units=units, normalization=normalization,verbose=1)

ev=lstm_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
tf.saved_model.save(lstm_ph,f'saved_model/vel/{name}')

Epoch 1/10
   7/5158 [..............................] - ETA: 5:31 - loss: 239.6912 - mean_absolute_error: 11.3693 - accuracy: 0.0000e+00

UnknownError: Graph execution error:

2 root error(s) found.
  (0) UNKNOWN:  RuntimeError: generator raised StopIteration
Traceback (most recent call last):

  File "C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\data_adapter.py", line 1050, in generator_fn
    yield x[i]

  File "D:\Users\Students\Daniele\Github\TesiRacingGame\AnacondaScripts\newDB\Generator.py", line 130, in __getitem__
    x,y=next(self.generator)

StopIteration


The above exception was the direct cause of the following exception:


Traceback (most recent call last):

  File "C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1035, in generat

In [11]:
tf.saved_model.save(lstm_ph,f'saved_model/vel/{name}')

NameError: name 'lstm_ph' is not defined

In [19]:
model=model_definitions.CNN_LSTM_Model(0.2, units=32,filters=32,kernel_size=1, normalization=False)
model.load_weights("saved_model\CNN_LSTM32_1_32_PosAssoluta")

In [20]:
model.summary()

Model: "CNN32_1LSTM32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 30, 5)]           0         
                                                                 
 conv1d (Conv1D)             (None, 30, 32)            192       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 30, 32)           0         
 )                                                               
                                                                 
 lstm_30 (LSTM)              (None, 32)                8320      
                                                                 
 dense_6 (Dense)             (None, 3)                 99        
                                                                 
Total params: 8,611
Trainable params: 8,611
Non-trainable params: 0
___________________________________________________

In [ ]:
import tf2onnx
import onnx
input_signature=(tf.TensorSpec([None,30,5],tf.float32,name='x'))
onnx_model,_=tf2onnx.converter.from_keras(model,input_signature,opset=9)

In [13]:
mlp_ph.predict(gen_test[0])

ValueError: in user code:

    File "C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\keras\engine\input_spec.py", line 216, in a